In [1]:
# Libraries

import sys
from bs4 import BeautifulSoup
import Cleaning as cln
from collections import defaultdict, Counter
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.multiprocessing import get
from bokeh.io import export_png
from bokeh.models import ColumnDataSource, TableColumn, DataTable
import datetime
import Exploration as exp
import gmaps
from importlib import reload
reload(cln)
reload(exp)
from langdetect import detect, DetectorFactory
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing
import nltk
import numpy as np
import os
import pandas as pd
import random
import scipy
import seaborn as sns
import sklearn as skl
from sklearn import manifold, metrics, model_selection, preprocessing, decomposition
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import label_binarize, LabelEncoder, OneHotEncoder, RobustScaler
from spellchecker import SpellChecker
import string
import time
import Transformer as mytransformer
reload(mytransformer)

pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_columns', 186)

%matplotlib widget
# %matplotlib inline

In [2]:
class progress_bar():
    
    def __init__(self, barLength = None, title = None, progress_lim = None, time_lim=None):
        self.first_time = datetime.datetime.now()
        if barLength is not None:
            self.barLength = 40
        else :
            self.barLength = barLength
        
        if title is None:
            self.title = ''
        else:
            self.title = title
            
        if progress_lim is not None:
            self.progress_lim = progress_lim
            self.step_progress = 1/self.progress_lim
            
            self.time_lim = datetime.timedelta(hours=10)
            self.step_time = 1
            
            self.lim_kind = 'progress'
        elif time_lim is not None:
            self.progress_lim = 0
            self.step_progress = 1
            
            self.time_lim = datetime.timedelta(seconds = time_lim)
            self.step_time = 1/time_lim
            
            self.lim_kind = 'time'
        else:
            self.progress_lim = 0.01
            self.time_lim = datetime.timedelta(hours=10)
            self.time_lim = 1
            self.lim_kind = 'progress'
            
        self.old_progress = 0
        self.old_timer = datetime.timedelta(seconds = 0)
        self.progress = 0
        self.timer = datetime.timedelta(seconds = 0)
        
    def set_progress(self, progress):
        self.old_progress = -1
        self.update_progress(progress)
        
    def reset(self, title=None, progress=None):
        self.first_time = datetime.datetime.now()
        self.old_progress = -1
        self.old_timer = datetime.timedelta(seconds = 0)
        if title is not None:
            self.title = title
        if progress is None:
            self.update_progress(0)
        else:
            self.update_progress(progress)
            
    def update_progress(self, progress):
        self.timer = datetime.datetime.now() - self.first_time
#         status = datetime.datetime.strptime(str(time_spent), "%H:%M:%S")
        status = str(self.timer).format("%H:%M:%.0S")
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Halt...\r\n"
        if progress >= 1 and self.old_progress < 1:
            self.old_progress = progress
            progress = 1
            status = "Done (" + status + ")\r\n"
        
        block = int(round(self.barLength*progress))
        if ((self.lim_kind == 'progress') & (progress >= self.old_progress + self.progress_lim)) |\
            ((self.lim_kind == 'time') & (self.timer >= self.old_timer + self.time_lim)):
            
            self.old_progress = (progress*self.step_progress - (progress*self.step_progress % 1))/self.step_progress
            self.old_timer = datetime.timedelta(
                seconds = 
                ( 
                    self.timer.total_seconds()*self.step_time
                    - (self.timer.total_seconds()*self.step_time % 1)
                )
                /self.step_time
            )
            
            text = "\rProgress {0}: [{1}] {2:.2f}% {3:s}".format( self.title, "#"*block + "-"*(self.barLength-block), progress*100, status)
            sys.stdout.write(text)
            sys.stdout.flush()
            
def save_df_as_image(df, path, width=None, height=None, width_policy = None, height_policy = None, b_index=False):
    df_copy = df.copy()
    if b_index:
        df_copy.insert(0, '', df_copy.index.values)
    
    if width is None:
        width = 280
    if height is None:
        height = 280
    if height_policy is None:
        height_policy = "auto"
    if width_policy is None:
        width_policy = "auto"
        
    source = ColumnDataSource(df_copy)
    df_columns = df_copy.columns.values

    columns_for_table = []
    
    for column in df_columns:
        columns_for_table.append(TableColumn(field=column, title=column))

    data_table = DataTable(source = source,
                           columns = columns_for_table,
                           width = width,
                           height = height,
                           height_policy = height_policy,
                           width_policy = width_policy,
                           index_position = None)
    export_png(data_table, filename = path)

## 1) Mise en forme du jeu de donné

Requet SQL :
``` mysql
SELECT Id, PostTypeId, ParentId, CreationDate, Score, ViewCount, Body, OwnerUserId, Title, Tags, AnswerCount, CommentCount, FavoriteCount, ClosedDate
        FROM posts
        WHERE ViewCount > 5000 AND (CreationDate BETWEEN '2020-01-01' AND '2020-01-31') AND AnswerCount > 0 AND CommentCount > 5 AND len(Tags) > 0
        ORDER BY CreationDate
```

Equilibré le nombre de post sur chaque mois et sur chaque année. 

In [3]:
# USING CSV file for data
# Method from : http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/prepare_data_2017.html

FILE_2019 = "./Data/Posts_2019.csv"
data_2019 = pd.read_csv(FILE_2019, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_2019, ' : ', os.stat(FILE_2019).st_size / 2**20, 'Mo, ', data_2019.shape)

FILE_2020 = "./Data/Posts_2020.csv"
data_2020 = pd.read_csv(FILE_2020, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_2020, ' : ', os.stat(FILE_2020).st_size / 2**20, 'Mo', data_2020.shape)

./Data/Posts_2019.csv  :  83.79751873016357 Mo,  (42393, 14)
./Data/Posts_2020.csv  :  37.11684703826904 Mo (17090, 14)


In [4]:
DataRaw = pd.concat([data_2019, data_2020], axis = 0, ignore_index=True)
DataRaw['CreationDate'] = DataRaw['CreationDate'].astype("datetime64")
DataRaw['ClosedDate'] = DataRaw['ClosedDate'].astype("datetime64")
DataRaw.describe(include = 'all', datetime_is_numeric=True)

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
count,5.948300e+04,59483.0,0.0,59483,59483.000000,5.948300e+04,59483,5.848000e+04,59483,59483,59483.000000,59483.000000,35825.000000,1995
unique,NaN,NaN,NaN,NaN,NaN,NaN,59483,NaN,59483,41177,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,<p>I was trying the todo-list exampl...,NaN,how to handle 500 error message with...,<flutter><dart>,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,392,NaN,NaN,NaN,NaN
mean,5.798659e+07,1.0,NaN,2019-09-14 07:00:44.380444928,9.164165,1.217933e+04,NaN,7.208163e+06,NaN,NaN,2.737606,2.017736,3.079749,2019-12-29 06:25:43.028070144
min,5.399238e+07,1.0,NaN,2019-01-01 00:48:55,-13.000000,5.001000e+03,NaN,9.100000e+01,NaN,NaN,1.000000,0.000000,0.000000,2019-01-01 10:07:16
25%,5.548793e+07,1.0,NaN,2019-04-03 05:52:03.500000,2.000000,6.081000e+03,NaN,3.497671e+06,NaN,NaN,1.000000,0.000000,1.000000,2019-04-30 06:37:09.500000
50%,5.742721e+07,1.0,NaN,2019-08-09 09:31:42,5.000000,7.899000e+03,NaN,7.726327e+06,NaN,NaN,2.000000,1.000000,2.000000,2019-11-08 14:20:51
75%,5.996404e+07,1.0,NaN,2020-01-29 09:27:59.500000,10.000000,1.211200e+04,NaN,1.091122e+07,NaN,NaN,3.000000,3.000000,3.000000,2020-07-27 01:43:07
max,6.552585e+07,1.0,NaN,2020-12-31 22:00:47,716.000000,1.579886e+06,NaN,1.571046e+07,NaN,NaN,51.000000,50.000000,203.000000,2021-12-03 04:33:51


In [5]:
DataRaw.head(10)

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,53992384,1,NaN,2019-01-01 00:48:55,6,9182,<p>I have 2 docker containers runnin...,5719544.0,How to access Docker container's int...,<docker><jenkins><sonarqube>,3,0,3.0,NaT
1,53992408,1,NaN,2019-01-01 00:55:46,2,12877,<p>I've tried all the suggestions I'...,1090414.0,Angular 5 - custom pipe 'could not b...,<angular><angular5>,4,8,NaN,NaT
2,53992415,1,NaN,2019-01-01 00:57:55,3,8801,"<p>I tried to get this to work, but ...",10853213.0,How to fill multidimensional array i...,<javascript><arrays><loops>,5,3,NaN,NaT
3,53992694,1,NaN,2019-01-01 02:30:21,22,15138,<p>I'm learning to make login functi...,8070090.0,What does netloc mean?,<python><networking><flask><flask-lo...,2,1,5.0,NaT
4,53992822,1,NaN,2019-01-01 03:17:23,8,6684,<p>Some users are getting this error...,23616.0,CompactToken validation failed 80049228,<microsoft-identity-platform>,1,2,2.0,NaT
5,53993007,1,NaN,2019-01-01 04:28:46,2,6121,<p>Am new with Next.js seeking your ...,9181738.0,Edit component property onClick Next.js,<javascript><material-ui><next.js>,1,0,NaN,NaT
6,53993016,1,NaN,2019-01-01 04:31:01,8,7710,<p>is that posible to create Nested ...,9661872.0,How to do multiple nested route with...,<javascript><vue.js><vuejs2><vue-com...,1,0,2.0,NaT
7,53993146,1,NaN,2019-01-01 05:14:41,1,9499,"<p><a href=""https://i.stack.imgur.co...",7404046.0,POSTMAN Request in form-data,<java><spring-mvc><postman>,2,1,NaN,NaT
8,53993154,1,NaN,2019-01-01 05:17:47,3,8726,<pre><code>ERROR in node_modules\@ng...,10299790.0,Getting error An expression of type ...,<datepicker><node-modules><angular7>,2,0,1.0,NaT
9,53993181,1,NaN,2019-01-01 05:24:09,28,56739,<p>DB:</p>\n\n<pre><code>$ mysql --v...,1481475.0,How to set correct MySQL JDBC timezo...,<mysql><spring-boot><hikaricp>,12,0,9.0,NaT


In [87]:
print(DataRaw.loc[7,'Title'])
print(DataRaw.loc[7,'Body'])

POSTMAN Request in form-data
<p><a href="https://i.stack.imgur.com/bGply.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/bGply.png" alt="enter image description here"></a>I have this method but I am not able to figure out which parameters should I pass in postman to execute this method. I sent the body in form-data of postman and content-Type is multipart/form-data.</p>

<p>Later I have to send the file as well that's why I am using form-data.</p>

<p>SearchString is coming out be empty in backend code.</p>

<pre><code>@RequestMapping(value = "/calendar", method = RequestMethod.POST)
protected FareCalendarResponse getCalendar(
        @RequestParam(value = "searchString", required = false) String searchString, HttpServletRequest httpRequest)
        throws Exception {
    CalendarRequest fareRequest = new CalenderRequest(searchString);
    return FareCalendarHandler.builder().httpRequest(httpRequest).request(fareRequest).build().getResponse();
}
</code></pre>



In [58]:
qual = ['Body', 'Title', 'Tags']
df = DataRaw[qual]
# df = df.sample(10000)

# def replace(row, feat, delimiter):
#     row[feat] = row[feat].replace(delimiter, '')
#     return row

# def tokkenize(row, feat, delimiter):
#     row[feat] = row.loc[:-1,feat].split(delimiter)
#     return row

df.loc[:,'Tags'] = df.loc[:,'Tags'].apply(lambda x: x.replace('<', '')).apply(lambda x: x[:-1].split('>'))
# len(df)
# df = df.apply(replace, args=('Tags', '<'), axis =1)
# df = df.apply(tokkenize, args=('Tags', '>'), axis =1)

# df.drop(['Body'], axis=1)
# df.loc[59482,'Tags']
df.describe(include = 'all', datetime_is_numeric=True)

,Body,Title,Tags
count,59483,59483,59483
unique,59483,59483,41177
top,<p>I have a SwiftUI view: </p>\n\n<p...,How to disable canActivate guard,"[flutter, dart]"
freq,1,1,392


In [35]:
df = DataRaw.copy()
soup = BeautifulSoup(df.loc[5,'Body'], 'html.parser')
text = soup.get_text()
text

'Am new with Next.js seeking your support to explain how to pass a new value for a property of a component. I used Material-UI library for styling.\nWhile am trying to change open property for a drawer component it always show for me [TypeError] open is read-only.\nconst drawer = (\n  <SwipeableDrawer open={drawerOpened}>\n    <div tabIndex={0} role="button">\n      {sideList}\n    </div>\n  </SwipeableDrawer>\n);\n\nconst handleClick = e => {\n  drawerOpened = !drawerOpened;\n  drawer.props.open = drawerOpened;\n  e.preventDefault();\n};\n\nconst Index = () => (\n  <div className={styles.root}>\n    <AppBar position="static">\n      <Toolbar>\n        <IconButton\n          className={styles.menuButton}\n          color="inherit"\n          aria-label="Menu"\n          onClick={handleClick}\n        >\n          <MenuIcon />\n        </IconButton>\n        <Typography variant="h6" color="inherit" className={styles.grow}>\n          Example\n        </Typography>\n        <Button color

In [37]:
selection = ['NN', 'NNS', 'NNP', 'NNPS']
punct_space = " "*len(string.punctuation)
digits_space = " "*len(string.digits)

text = text.translate(str.maketrans(string.punctuation, punct_space, ''))
text = text.translate(str.maketrans(string.digits, digits_space, ''))
while '  ' in text:
        text = text.replace('  ', ' ')
        
tokens = nltk.word_tokenize(text)
tagged = nltk.pos_tag(tokens)
tagged

# vec = [token.lower() for (token, tag) in tagged if (tag in selection) & (len(token) > 1)]
# vec

[('Am', 'NNP'),
 ('new', 'JJ'),
 ('with', 'IN'),
 ('Next', 'JJ'),
 ('js', 'NN'),
 ('seeking', 'VBG'),
 ('your', 'PRP$'),
 ('support', 'NN'),
 ('to', 'TO'),
 ('explain', 'VB'),
 ('how', 'WRB'),
 ('to', 'TO'),
 ('pass', 'VB'),
 ('a', 'DT'),
 ('new', 'JJ'),
 ('value', 'NN'),
 ('for', 'IN'),
 ('a', 'DT'),
 ('property', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('component', 'NN'),
 ('I', 'PRP'),
 ('used', 'VBD'),
 ('Material', 'NNP'),
 ('UI', 'NNP'),
 ('library', 'NN'),
 ('for', 'IN'),
 ('styling', 'VBG'),
 ('While', 'IN'),
 ('am', 'VBP'),
 ('trying', 'VBG'),
 ('to', 'TO'),
 ('change', 'VB'),
 ('open', 'JJ'),
 ('property', 'NN'),
 ('for', 'IN'),
 ('a', 'DT'),
 ('drawer', 'NN'),
 ('component', 'NN'),
 ('it', 'PRP'),
 ('always', 'RB'),
 ('show', 'VBP'),
 ('for', 'IN'),
 ('me', 'PRP'),
 ('TypeError', 'NNP'),
 ('open', 'JJ'),
 ('is', 'VBZ'),
 ('read', 'VBN'),
 ('only', 'RB'),
 ('const', 'JJ'),
 ('drawer', 'NN'),
 ('SwipeableDrawer', 'NNP'),
 ('open', 'JJ'),
 ('drawerOpened', 'VBD'),
 ('div', 'JJ'),

In [105]:
DetectorFactory.seed = 0
detect(text)

'ca'

In [101]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Blaise-
[nltk_data]     Maël\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Blaise-Maël\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
ps = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

tokens = nltk.word_tokenize(text)
tokens = [ps.stem(tok) for tok in tokens]

tagged = nltk.pos_tag(tokens)
tagged

[('i', 'NNS'),
 ('have', 'VBP'),
 ('thi', 'VBN'),
 ('method', 'NN'),
 ('but', 'CC'),
 ('i', 'NN'),
 ('am', 'VBP'),
 ('not', 'RB'),
 ('abl', 'JJ'),
 ('to', 'TO'),
 ('figur', 'VB'),
 ('out', 'RP'),
 ('which', 'WDT'),
 ('paramet', 'NN'),
 ('should', 'MD'),
 ('i', 'VB'),
 ('pass', 'NN'),
 ('in', 'IN'),
 ('postman', 'NN'),
 ('to', 'TO'),
 ('execut', 'VB'),
 ('thi', 'JJ'),
 ('method', 'NN'),
 ('.', '.'),
 ('i', 'VB'),
 ('sent', 'VBD'),
 ('the', 'DT'),
 ('bodi', 'NN'),
 ('in', 'IN'),
 ('form-data', 'NN'),
 ('of', 'IN'),
 ('postman', 'NN'),
 ('and', 'CC'),
 ('content-typ', 'NN'),
 ('is', 'VBZ'),
 ('multipart/form-data', 'JJ'),
 ('.', '.'),
 ('later', 'RB'),
 ('i', 'RB'),
 ('have', 'VBP'),
 ('to', 'TO'),
 ('send', 'VB'),
 ('the', 'DT'),
 ('file', 'NN'),
 ('as', 'RB'),
 ('well', 'RB'),
 ('that', 'IN'),
 ("'s", 'VBZ'),
 ('whi', 'NN'),
 ('i', 'NN'),
 ('am', 'VBP'),
 ('use', 'RB'),
 ('form-data', 'JJ'),
 ('.', '.'),
 ('searchstr', 'NN'),
 ('is', 'VBZ'),
 ('come', 'VBN'),
 ('out', 'RP'),
 ('be', 'VB

In [31]:
selection = ['NN', 'NNS', 'NNP', 'NNPS']
punct_space = " "*len(string.punctuation)
digits_space = " "*len(string.digits)

ps = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

text = text.translate(str.maketrans(string.punctuation, punct_space, ''))
text = text.translate(str.maketrans(string.digits, digits_space, ''))
while '  ' in text:
        text = text.replace('  ', ' ')
        
tokens = nltk.word_tokenize(text)
tagged = nltk.pos_tag(tokens)
tagged

vec = [token.lower() for (token, tag) in tagged if (tag in selection) & (len(token) > 1)]
vec = [ps.stem(tok) for tok in vec]
vec

['method',
 'paramet',
 'postman',
 'method',
 'bodi',
 'data',
 'postman',
 'content',
 'type',
 'form',
 'data',
 'file',
 'form',
 'data',
 'searchstr',
 'code',
 'valu',
 'calendar',
 'method',
 'requestmethod',
 'post',
 'farecalendarrespons',
 'getcalendar',
 'requestparam',
 'valu',
 'searchstr',
 'string',
 'httpservletrequest',
 'httprequest',
 'except',
 'calendarrequest',
 'calenderrequest',
 'return',
 'farecalendarhandl',
 'builder',
 'httprequest',
 'httprequest',
 'request',
 'build',
 'getrespons']

In [103]:
selection = ['NN', 'NNS', 'NNP', 'NNPS']

vec = [token.lower() for (token, tag) in tagged if tag in selection]
vec

['method',
 'parameters',
 'postman',
 'method',
 'body',
 'data',
 'postman',
 'content',
 'type',
 'form',
 'data',
 'file',
 'form',
 'data',
 'searchstring',
 'code',
 'value',
 'calendar',
 'method',
 'requestmethod',
 'post',
 'farecalendarresponse',
 'getcalendar',
 'requestparam',
 'value',
 'searchstring',
 'string',
 'httpservletrequest',
 'httprequest',
 'exception',
 'calendarrequest',
 'calenderrequest',
 'return',
 'farecalendarhandler',
 'builder',
 'httprequest',
 'httprequest',
 'request',
 'build',
 'getresponse']

In [65]:
spell = SpellChecker()
[spell.correction(word) for word in vec]

['docker',
 'containers',
 'mac',
 'host',
 'container',
 'docker',
 'hub',
 'container',
 'docker',
 'hub',
 'containers',
 'access',
 'jenkins',
 'host',
 '//localhost:8080/',
 'access',
 'sonarqube',
 '//localhost:9000/',
 'jenkins',
 'container',
 'docker',
 'ad',
 'up',
 'jenkins/jenkins',
 'sonarqube',
 'container',
 'docker',
 'ad',
 'up',
 'sonarqube',
 'container',
 'communicate',
 'ip',
 'address',
 'container',
 'container',
 'ip',
 'address',
 'container',
 'docker',
 'inspect',
 'format',
 'container_name_or_id',
 'ip',
 'address',
 'jenkins',
 'container',
 'sonarqube',
 'container',
 'access',
 'jenkins',
 'container',
 'host',
 '//172.17.0.2:8080',
 'request',
 'timeout',
 'thing',
 'access',
 'sonarqube',
 'container',
 'host',
 '//172.17.0.3:9000',
 'behavior',
 'access',
 'container',
 'host',
 'ip',
 'address',
 'container',
 'jenkins',
 'access',
 'container',
 'sonarqube',
 'ip',
 'address']

In [8]:
# df.loc[0,'Title']
soup = BeautifulSoup(df.loc[65,'Title'], 'html.parser')
title = soup.get_text()
title

'Puppeteer Error: Chromium revision is not downloaded'

In [6]:
def set_str(row, bar=None, lenght = None, lemmatizer = None, stem = None):
    errors_counts = 0
    selection = ['NN', 'NNS', 'NNP', 'NNPS']
    stop_words = set(nltk.corpus.stopwords.words('english'))
    
    punct_space = " "*len(string.punctuation)
    digits_space = " "*len(string.digits)

    if lenght is not None:
        progress = row.name / lenght
    else:
        progress = 1
    
    vec_title = []
    vec_body = []
        
    title = BeautifulSoup(row['Title'], 'html.parser').get_text()
    body = BeautifulSoup(row['Body'], 'html.parser').get_text()
    
    title = title.translate(str.maketrans(string.punctuation, punct_space,''))
    body = body.translate(str.maketrans(string.punctuation, punct_space,''))

    title = title.translate(str.maketrans(string.digits, digits_space,''))
    body = body.translate(str.maketrans(string.digits, digits_space,''))

    while '  ' in title:
        title = title.replace('  ', ' ')

    while '  ' in body:
        body = body.replace('  ', ' ')
                
    try:
        if detect(body) == 'en':

            # Suppression des mots qui ne sont pas des noms
            tok_title = nltk.word_tokenize(title)
            tok_body = nltk.word_tokenize(body)

            tag_title = nltk.pos_tag(tok_title)
            tag_body = nltk.pos_tag(tok_body)

            vec_title = [lemmatizer.lemmatize(token.lower()) for (token, tag) in tag_title if (tag in selection) & (len(token) > 1)& (token not in stop_words)]
            vec_body = [lemmatizer.lemmatize(token.lower()) for (token, tag) in tag_body if (tag in selection) & (len(token) > 1) & (token not in stop_words)]
        
    except:
        print("\nThis body and langag detect error: \n {0:s}\n {1:s}\n".format(row['Body'], body))
        errors_counts += 0
        
    row['Title'] = vec_title
    row['Body'] = vec_body

    if bar is not None:
        bar.update_progress(progress)
        
    return row

In [7]:
DetectorFactory.seed = 0
# nltk.download('stopwords')
# nltk.download('wordnet')

ps = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

ProgressBar().register()
ddata = dd.from_pandas(DataRaw, npartitions=4*multiprocessing.cpu_count())
with ProgressBar():
    DataModel = ddata.map_partitions(
        lambda df: df.apply(
            set_str,
            args = (None, len(df), lemmatizer), axis=1)
    ).compute(scheduler='processes')

DataModel = DataModel[DataModel.astype(str)['Title'] != '[]']

DataModel.describe(include='all', datetime_is_numeric=True)

[########################################] | 100% Completed | 11min 15.9s
[########################################] | 100% Completed | 11min 16.0s


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
count,5.933800e+04,59338.0,0.0,59338,59338.000000,5.933800e+04,59338,5.834200e+04,59338,59338,59338.000000,59338.000000,35732.000000,1984
unique,NaN,NaN,NaN,NaN,NaN,NaN,59336,NaN,58025,41087,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,"[package, funding, react, project, a...",NaN,[value],<flutter><dart>,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,21,389,NaN,NaN,NaN,NaN
mean,5.798667e+07,1.0,NaN,2019-09-14 07:06:16.765984768,9.141528,1.217382e+04,NaN,7.207210e+06,NaN,NaN,2.737116,2.016364,3.073827,2019-12-29 09:08:18.023689472
min,5.399238e+07,1.0,NaN,2019-01-01 00:48:55,-13.000000,5.001000e+03,NaN,9.100000e+01,NaN,NaN,1.000000,0.000000,0.000000,2019-01-01 10:07:16
25%,5.548859e+07,1.0,NaN,2019-04-03 06:39:02.750000128,2.000000,6.081000e+03,NaN,3.497671e+06,NaN,NaN,1.000000,0.000000,1.000000,2019-04-30 06:16:36
50%,5.742716e+07,1.0,NaN,2019-08-09 09:28:44,5.000000,7.898000e+03,NaN,7.724838e+06,NaN,NaN,2.000000,1.000000,2.000000,2019-11-08 12:01:00.500000
75%,5.996326e+07,1.0,NaN,2020-01-29 08:39:18.249999872,10.000000,1.211200e+04,NaN,1.091108e+07,NaN,NaN,3.000000,3.000000,3.000000,2020-07-27 18:00:43.500000
max,6.552585e+07,1.0,NaN,2020-12-31 22:00:47,716.000000,1.579886e+06,NaN,1.571046e+07,NaN,NaN,51.000000,50.000000,203.000000,2021-12-03 04:33:51


In [20]:
bar = progress_bar(10, 'treatement', 1e-2)
# bar = progress_bar(10, 'treatement', time_lim=1)
bar.set_progress(0)

lemmatizer = nltk.stem.WordNetLemmatizer()
DataModel = DataRaw.apply(set_str, args=(bar, len(DataRaw), lemmatizer), axis = 1)
DataModel = DataModel[DataModel.astype(str)['Title'] != '[]']
bar.update_progress(1)

DataModel.describe(include='all', datetime_is_numeric=True)

Progress treatement: [#---------------------------------------] 2.00% 0:00:30.412261This body and langag detect error: 
 <p>Using the <a href="http://docs.groovy-lang.org/latest/html/documentation/#_named_parameters_2" rel="noreferrer">documented mixture of named and positional arguments</a> (or args with default values for that matter), how to invoke a method without specifying any named param?</p>

<p>I'm trying to extend existing shared method without breaking everybody else's code, and this approach looked promising, but the following example fails:</p>

<pre><code>​def test(Map args, some, thing='default value'){
    "$some $thing";
}

//good - adding any named parameter works
//test('yet', 'another good', anything:'notneeded');

//but not specifying named parameter fails
test('this', 'fails');​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Progress treatement: [###########-----------------------------] 27.00% 0:06:40.713678This body and langag detect error: 
 <pre><code>   === npm audit security report ===                        

┌───────────────────────────────────────────────────────────────────┐
│                                Manual Review                      │
│      Some vulnerabilities require your attention to resolve       │
│                                                                   │
│  Visit https://go.npm.me/audit-guide for additional guidance      │
└───────────────────────────────────────────────────────────────────┘
┌───────────────┬───────────────────────────────────────────────────┐
│ High          │ Arbitrary File Overwrite                          │
├───────────────┼───────────────────────────────────────────────────┤
│ Package       │ tar                                               │
├───────────────┼───────────────────────────────────────────────────┤
│ Patched in    │ &gt;=4.4.2       

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
count,5.933300e+04,59333.0,0.0,59333,59333.000000,5.933300e+04,59333,5.833600e+04,59333,59333,59333.000000,59333.000000,35729.000000,1984
unique,NaN,NaN,NaN,NaN,NaN,NaN,382,NaN,58030,41081,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,[value],<flutter><dart>,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,52388,NaN,20,390,NaN,NaN,NaN,NaN
mean,5.798683e+07,1.0,NaN,2019-09-14 07:21:28.484822784,9.142720,1.217476e+04,NaN,7.206862e+06,NaN,NaN,2.737296,2.016028,3.074029,2019-12-28 23:47:03.532257792
min,5.399238e+07,1.0,NaN,2019-01-01 00:48:55,-13.000000,5.001000e+03,NaN,9.100000e+01,NaN,NaN,1.000000,0.000000,0.000000,2019-01-01 10:07:16
25%,5.548861e+07,1.0,NaN,2019-04-03 06:39:59,2.000000,6.081000e+03,NaN,3.497671e+06,NaN,NaN,1.000000,0.000000,1.000000,2019-04-30 06:16:36
50%,5.742724e+07,1.0,NaN,2019-08-09 09:33:40,5.000000,7.898000e+03,NaN,7.723946e+06,NaN,NaN,2.000000,1.000000,2.000000,2019-11-07 12:32:59.500000
75%,5.996338e+07,1.0,NaN,2020-01-29 08:47:36,10.000000,1.211200e+04,NaN,1.091062e+07,NaN,NaN,3.000000,3.000000,3.000000,2020-07-26 21:59:38
max,6.552585e+07,1.0,NaN,2020-12-31 22:00:47,716.000000,1.579886e+06,NaN,1.571046e+07,NaN,NaN,51.000000,50.000000,203.000000,2021-12-03 04:33:51


In [163]:
multiprocessing.cpu_count()

8

In [8]:
DataModel['Tags'] = DataModel['Tags'].apply(lambda x: x.replace('<', '')).apply(lambda x: x[:-1].split('>'))

In [9]:
DataModel.head(10)

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,53992384,1,NaN,2019-01-01 00:48:55,6,9182,"[container, mac, host, container, je...",5719544.0,"[access, docker, container, ip, addr...","[docker, jenkins, sonarqube]",3,0,3.0,NaT
1,53992408,1,NaN,2019-01-01 00:55:46,2,12877,"[suggestion, template, parse, pipe, ...",1090414.0,"[custom, pipe, error]","[angular, angular5]",4,8,NaN,NaT
2,53992415,1,NaN,2019-01-01 00:57:55,3,8801,"[outer, loop, stop, iteration, every...",10853213.0,"[array, javascript]","[javascript, arrays, loops]",5,3,NaN,NaT
4,53992822,1,NaN,2019-01-01 03:17:23,8,6684,"[user, error, microsoft, sign, order...",23616.0,"[compacttoken, validation]",[microsoft-identity-platform],1,2,2.0,NaT
5,53993007,1,NaN,2019-01-01 04:28:46,2,6121,"[am, j, support, value, property, co...",9181738.0,"[edit, component, property, next, j]","[javascript, material-ui, next.js]",1,0,NaN,NaT
6,53993016,1,NaN,2019-01-01 04:31:01,8,7710,"[nested, route, something, user, pro...",9661872.0,"[route, vue, j]","[javascript, vue.js, vuejs2, vue-com...",1,0,2.0,NaT
7,53993146,1,NaN,2019-01-01 05:14:41,1,9499,"[method, parameter, postman, method,...",7404046.0,"[postman, request, form, data]","[java, spring-mvc, postman]",2,1,NaN,NaT
8,53993154,1,NaN,2019-01-01 05:17:47,3,8726,"[error, module, bootstrap, datepicke...",10299790.0,"[error, expression, type, void, trut...","[datepicker, node-modules, angular7]",2,0,1.0,NaT
9,53993181,1,NaN,2019-01-01 05:24:09,28,56739,"[db, version, mysql, ver, distrib, o...",1481475.0,"[mysql, jdbc, timezone, spring, boot...","[mysql, spring-boot, hikaricp]",12,0,9.0,NaT
10,53993252,1,NaN,2019-01-01 05:41:30,9,5754,"[nuxt, vuetify, carousel, component,...",1592380.0,"[image, path, folder, nuxt]","[javascript, vue.js, webpack, nuxt.js]",1,8,2.0,NaT


In [31]:
feat = 'Body'

lookedfor = [['error']]
# lookedfor = ['abcdefghijklmonpqrstuvwxyz']
# lookedfor = [string.punctuation]
def check_words_list(column, lookedfor):
    return list(set(column) & set(lookedfor))
    
test = pd.DataFrame(DataModel[feat].apply(check_words_list, args=(lookedfor)))
test = test[test.astype(str)[feat] != '[]']
test

,Body
4,[error]
8,[error]
9,[error]
10,[error]
13,[error]
...,...
59476,[error]
59477,[error]
59478,[error]
59479,[error]


## 2) Analyse du dataset

### 2.A) Analyse univariée 

In [38]:
DataModel.plot(x='Id', y='AnswerCount', kind='scatter') 

<AxesSubplot:xlabel='Id', ylabel='AnswerCount'>

In [39]:
DataModel.plot(x='Id', y='ViewCount')

<AxesSubplot:xlabel='Id'>

In [147]:
# DataRaw.loc[0,'Body']

In [10]:
top = 50
count, words = exp.most_common_words(DataModel['Body'], ', ', top)
pd.DataFrame([words, count])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,error,file,java,name,class,data,import,user,code,com,module,value,app,version,http,j,id,package,type,python,function,return,test,line,string,component,image,div,project,const,core,log,json,text,service,path,node,server,request,model,message,py,application,android,list,option,api,button,react,default
1,69881,59499,53710,49113,47299,43051,42930,40033,36739,36173,36039,35539,34792,32905,32408,30241,28592,28308,27597,25560,25535,25371,24270,23169,21566,21561,21050,20106,19765,19315,19297,19136,18609,18445,18046,18004,17702,17345,17080,16942,16870,16515,16508,16383,16332,15798,15311,15303,15103,14917


On remarque ici un problème, ce sont les tags de type : 'c++' dans le texte que l'on ne pourras pas détecter car on veut supprimer la ponctuation et les lettres seules.
On estime que si se sont des tags imports nous les retrouverons dans les tags fournit par les utilisateurs.

In [33]:
fig = plt.figure(figsize=(10,9))

plt.hist(x=words, weights=count, bins = top)
plt.title('The top {:d} of most frequent words'.format(top))
plt.xticks(rotation=90)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.B) Analyse bivariée 

## 3) Réduction dimensionnelle & Visualisation

### 3.A) Text Fréquency - Inverse Document Frequency (tf-idf)

On peut transformer un texte en ses features soit :
- En utilisant une représentation de comptage creuse - fréquence d’apparition du mot dans un document, ou vecteur tf-idf d’un document, etc.
- En utilisant une représentation type word2vec dense - dans laquelle le mot possède une représentation dans un espace qui le positionne en fonction des mots adjacents

Ici on veut carractériser les textes par un tags on veut donc rechercher des mots-clef. A cette fin une représentation de comptage creuse semble plus pertinente.

#### 3.A.1) Réduction ds données par tf-idf

In [30]:
def f_return(text):
    return text

df = DataModel.sample(10000)

corpora = df['Title'] + df['Body']

 # print('Corpora :', corpus)
 # print(len(corpora),'\n')

vectorizer = skl.feature_extraction.text.TfidfVectorizer(
    token_pattern=None,
    analyzer = 'word',
    preprocessor = f_return,
    tokenizer = f_return
    )
X = vectorizer.fit_transform(corpora)
 # print('TF-iDF : ', vectorizer.get_feature_names())
DataBow = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
DataBow.describe()

,a,aa,aaa,aaaa,aaaaa,aaaaaa,aaaaaaaa,aaaaaaaaaaa,aaaaaaaaaai,aaaabjaaaadpexyunejxjygrgyobimgcwy,aaaarrrrrhhhhhhhhhhhhhhhh,aaab,aaabag,aaabhqzg,aaacxaibbkcmgcgaszaqgicaaciqgsi,aaaee,aaaohaaaabmaaacm,aaarch,aab,aabbcc,aabd,aaby,aac,aacad,aacqhoh,aactor,aad,aadappkey,aadarshvelu,aadb,aadc,aadclientid,aadinstance,aadsts,aae,aaeb,aaeccf,aaf,aafea,aafhrszjtwpe,aagin,aagxvy,aamir,aamr,aan,aantal,aantalpersonen,aantbd,aapcs,aapl,aapt,aaptcompiler,aaq,aaqskzjrgabaqaaa,aaqskzjrgabaqaaaqabaad,aaqskzjrgabaqeasabiaad,aar,aarch,aaron,aaronksaunders,aat,aazgjwfdmvna,ab,aba,abaamaaaabaaeaakacaaqaaaabaaafukadaaqaaaabaaacwaaaaaaaaaagaqmaawaaaaeabgaaaroabqaaaaeaaaeearsabqaaaaeaaaemasgaawaaaaeaagaaageabaaaaaeaaaeuagiabaaaaaeaacfraaaaaaaaaegaaaabaaaasaaaaah,abalari,abandon,abap,abb,abbe,abbee,abbf,abby,abc,abcb,abcbot,abcd,abcdabcdabcdabcd,abcde,abcdef,abcdefg,abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz,abciximab,abcjs,abclist,abcmeta,abcrad,abctest,abd,abdd,abddhcgbnmcysz,abdelhamedabdin,abdfecca,...,├───create,├───list,├───login,├────debug,├────random,├────ressource,├────────────┼────────────────────────┼───────────────┼─────────────┤,├──────────┼────┼─────────┼──────┼───────┼────────┼─────────┼────────┼─────┼───────────┼───────┼──────────┤,├──────────┼────┼─────────┼──────┼─────┼─────────┼─────────┼────────┼─────┼────────┼───────┼──────────┤,════════,════════════════════════════════════════════════════════════════════════════════════════════════════,╞══,╞════════════════════════════════════════════════════,╞═════════════════════════════════════════════════════════,╞═══════════════════════════════════════════════════════════,████████████████████████████████,░░░░░░░░░░░░░░░░,◀◀◀,◀◀◀———————,◀◀◀——————————————,◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤,⚠️,⚡️,✔️,⸮ets,する,する。,でエラーが発生した場合はリポートファイルがないことを許容する。,にステータスを通知する。,にバージョンアップで以下のものが使えなくなったので一旦コメントアウト。,に投稿する。,ので、,の起動や終了をするようにした。,をファイルに書き出しておく。,アイコン,サブスクリプションキーは変更してください,スクリプトで,タスクの入力と出力の両方とも更新がなければ,タスクを実行し、実行後に成功か失敗かを,タスク実行前に,テスト失敗時にも結果を保存するように,ファイル名を変更,フォルダ中の画像をai解析し、ファイルのリネームを行います。,使い方,使用,参考,善用,大秘訣，確保在家工作時的工作效率,失敗時の,実際に送信する際にはここのコメントアウトを外す。,居家办公的八大妙招,很奇怪,成功時の,我确认好多次我没打错了,文章の空白をファイル名用にアンダーバーに修正,有问题,機能概要,注意,状態を通知し、タスク完了後に、タスクの結果に応じて成功か失敗の状態を通知する。,社内では,経過時間を出力,载入程辑包：,载入需要的程辑包：data,载入需要的程辑包：lubridate,载入需要的程辑包：performance,载入需要的程辑包：psych,载入需要的程辑包：rio,送信する,错误,관련된,도움말,수행과,업무,재택근무,취소됨,：fix,｢wdm｣,｢wds｣,�a��,�n�w�,�pk,�yi�ޞ�b��,�˶x��,��,��d,���,����,����������,🖥️,🙋‍♂️i,🤦🏻‍♂️,🤷‍♂️,󈚧󈚨
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0000

In [16]:
# DataBow.select_dtypes(exclude=['datetime64'])

#### 3.A.2) ACP

In [35]:
Values = DataBow.values
features = DataBow.columns

pca = decomposition.PCA(n_components = len(features))
pca.fit(Values)

#seuils pour test des bâtons brisés 
bs = 1/np.arange(len(features), 0, -1) 
bs = np.cumsum(bs) 
bs = bs[::-1]

eigen_values = pca.explained_variance_
inertias = pca.explained_variance_ratio_*100

val = []
Total_inertia = []
selected_axes = []

cumsum = 0
for i in range(len(features)) :
    if bs[i] <= eigen_values[i]:
        val.append(True)
        cumsum += inertias[i]
        selected_axes.append('F{:d}'.format(i))
    else :
        val.append(False)
    Total_inertia.append(cumsum)

print(pd.DataFrame({'Eigen values': eigen_values,
                    'Seuils': bs, 
                    'Validation' : val, 
                    'Valid. Inertia' : Total_inertia}))

print(selected_axes)

exp.display_scree_plot(pca)
plt.axhline(90, color='red', linestyle='dashed', linewidth=2)
plt.text(1, 92, 'Seuil à 90%')
plt.axhline(Total_inertia[-1], color='blue', linestyle='dashed', linewidth=2)
plt.text(1, Total_inertia[-1]+2, 'Seuil à {:.0f}%'.format(Total_inertia[-1]))
plt.show(block=False)

      Eigen values    Seuils  Validation  Valid. Inertia
0     6.985845e-03  9.257802       False               0
1     6.367974e-03  8.257802       False               0
2     6.053243e-03  7.757802       False               0
3     5.967752e-03  7.424469       False               0
4     5.684234e-03  7.174469       False               0
...            ...       ...         ...             ...
5882  3.632402e-37  0.000850       False               0
5883  2.285375e-37  0.000680       False               0
5884  1.088089e-37  0.000510       False               0
5885  4.137497e-38  0.000340       False               0
5886  6.562572e-39  0.000170       False               0

[5887 rows x 4 columns]
[]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
n_view = 2

class DataFrameTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_comp=None):          
        if n_comp is not None:
            self.n_comp = n_comp
        else :
            self.n_comp = None
        
        self.std_trans = None
        self.acp_trans = None
        
        self.axis = []

    def fit(self, df):
        self.features = df.columns
        
        if self.n_comp is None:
            self.n_comp = len(self.features)
            
        for i in range(self.n_comp):
            self.axis.append('F%d' % i)
            
        X = df.values
            
#         self.std_trans = preprocessing.MinMaxScaler().fit(X)
#         X_trash = self.std_trans.transform(X)
        self.acp_trans = decomposition.PCA(n_components=self.n_comp).fit(X)
           
        return self
    
    def transform(self, df):
        X = df.values
        
#         X = self.std_trans.transform(X)
        X = self.acp_trans.transform(X)
        
        result = pd.DataFrame(X, columns = self.axis)
        return result

    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)
    
    def inverse_transform(self, df):
        X = df.values
        X = self.acp_trans.inverse_transform(X)
#         X = self.std_trans.inverse_transform(X)
        
        result = pd.DataFrame(X, columns = self.features)
        return result

#### 3.A.3) t-SNE

In [14]:
n_feat = 2559

data_pca = DataBow.copy()

features = data_pca.columns
trans_pca = DataFrameTransformer(n_feat)

data_pca = trans_pca.fit_transform(data_pca)

Values = data_pca.values
features = data_pca.columns

pca = decomposition.PCA(n_components=n_view).fit(Values)
X_PCA = pca.transform(Values)

plt.ioff()
fig3D = plt.figure(figsize=(7,6))
if n_view == 3:
    ax = fig3D.gca(projection='3d')
    ax.scatter(X_PCA[:,0], X_PCA[:,1], X_PCA[:,2])
    ax.set_xlabel('F0', fontsize=10)
    ax.set_ylabel('F1', fontsize=10)
    ax.set_zlabel('F2', fontsize=10)
elif n_view == 2:
    ax = fig3D.gca()
    ax.scatter(X_PCA[:,0], X_PCA[:,1])
    ax.set_xlabel('F0', fontsize=10)
    ax.set_ylabel('F1', fontsize=10)
else :
    print('No Draw')

plt.show(block=False)

data_pca.describe()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56,F57,F58,F59,F60,F61,F62,F63,F64,F65,F66,F67,F68,F69,F70,F71,F72,F73,F74,F75,F76,F77,F78,F79,F80,F81,F82,F83,F84,F85,F86,F87,F88,F89,F90,F91,F92,...,F2466,F2467,F2468,F2469,F2470,F2471,F2472,F2473,F2474,F2475,F2476,F2477,F2478,F2479,F2480,F2481,F2482,F2483,F2484,F2485,F2486,F2487,F2488,F2489,F2490,F2491,F2492,F2493,F2494,F2495,F2496,F2497,F2498,F2499,F2500,F2501,F2502,F2503,F2504,F2505,F2506,F2507,F2508,F2509,F2510,F2511,F2512,F2513,F2514,F2515,F2516,F2517,F2518,F2519,F2520,F2521,F2522,F2523,F2524,F2525,F2526,F2527,F2528,F2529,F2530,F2531,F2532,F2533,F2534,F2535,F2536,F2537,F2538,F2539,F2540,F2541,F2542,F2543,F2544,F2545,F2546,F2547,F2548,F2549,F2550,F2551,F2552,F2553,F2554,F2555,F2556,F2557,F2558
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,...,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,6.536265e-18,-3.316791e-18,-5.289887e-18,3.166911e-18,7.755667e-18,3.792106e-18,2.242130e-18,-3.745919e-20,-1.176576e-18,2.920580e-18,1.332528e-18,5.604718e-18,2.113761e-18,1.782970e-18,-2.956749e-18,-1.437110e-18,-4.358666e-18,1.360630e-18,-1.380666e-18,-5.967449e-20,-3.423236e-18,1.925717e-18,-9.242249e-18,3.051552e-18,-2.287753e-18,3.472396e-18,-3.072304e-18,2.138610e-18,-3.193366e-18,5.373154e-18,6.371206e-19,-8.047165e-19,-3.294023e-18,-2.917420e-18,-3.538619e-18,-1.708009e-18,-2.702786e-18,-8.342285e-19,-6.388987e-19,1.546354e-18,-3.228949e-18,-5.278937e-18,3.798611e-19,-5.398459e-19,-2.899634e-18,-1.507561e-18,-6.204239e-19,2.3

2566 features selected

In [15]:
Values = DataBow.values
features = DataBow.columns

pca = decomposition.PCA(n_components = len(features))
Values = pca.fit_transform(Values)

tsne = manifold.TSNE(n_components = n_view, perplexity=100, n_iter=1000, learning_rate=200, init='pca')
X_TSNE_classique = tsne.fit_transform(Values)

if n_view == 3:
    fig3D = plt.figure(figsize=(7,6))
    ax = fig3D.gca(projection='3d')
    ax.scatter(X_TSNE_classique[:,0], X_TSNE_classique[:,1], X_TSNE_classique[:,2])
    ax.set_xlabel('F0', fontsize=10)
    ax.set_ylabel('F1', fontsize=10)
    ax.set_zlabel('F2', fontsize=10)

elif n_view == 2:
    fig = plt.figure(figsize=(6, 4))
    ax = fig.gca()
    ax.scatter(X_TSNE_classique[:,0], X_TSNE_classique[:,1])
    ax.set_xlabel('F0', fontsize=10)
    ax.set_ylabel('F1', fontsize=10)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3.B) Word2vec

### 3.C) Word2doc

## 3) Keywords extraction

On veut ici déterminer les différents sujets du corpus. Pour chaque document on veut déterminer les sujets abordés.

### 3.A) Latent Dirichlet allocation (LDA)
C’est une méthode non-supervisée générative qui se base sur les hypothèses suivantes :

- Chaque document du corpus est un ensemble de mots sans ordre (bag-of-words) ;
- Chaque document m aborde un certain nombre de thèmes dans différentes proportions qui lui sont propres p(θm)
- Chaque mot possède une distribution associée à chaque thème p(ϕk). On peut ainsi représenter chaque thème par une probabilité sur chaque mot.
- zn représente le thème du mot wn

Puisque l'on a accès uniquement aux documents, on doit déterminer quels sont les thèmes, les distributions de chaque mot sur les thèmes, la fréquence d’apparition de chaque thème sur le corpus. <br>

A titre d’exemple, voici une liste des hypothèses supplémentaires qui mènent à une modélisation plus riche :

- La distribution des mots sur les thèmes évoluent avec le temps. Ce qui signifie qu’il faut créer une séquence de distribution pour chaque thème qui permet de modéliser l’évolution du thème dans le temps.
- Certains thèmes sont plus proches que d’autres. L’hypothèse d’utiliser la distribution de Dirichlet considère que les différents thèmes sont complètement indépendants alors qu’en réalité certains thèmes ont en général plus de chances d’apparaître ensemble.

In [16]:
max_iter = 10
no_top_words = 10

search_params = {
    'n_components': [5,10, 15, 20, 25, 30],
    'learning_decay': [.5, .7, .9]
}

# data_classique = data_classique.select_dtypes(exclude=['datetime64'])
Values = DataBow.values
features_names = DataBow.columns

lda = skl.decomposition.LatentDirichletAllocation(
#     n_components = n_topics,
    max_iter = max_iter, 
    learning_method = 'online',
    learning_offset = 50,
#     learning_decay = 0.7,
    random_state = 0,
    verbose = 0
)

model = GridSearchCV(
    cv = None,
    error_score = 'raise',
    estimator = lda,
    param_grid = search_params,
#     scoring = None,
    verbose = 2
)

model.fit(Values)

print('\n')
for topic_idx, topic in enumerate(model.best_estimator_.components_):
    print('Topic %d:' % (topic_idx))
    print(' '.join([features_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

print('\n')
# Log Likelyhood: Higher the better
print("Log Likelihood: ", model.best_estimator_.score(Values))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", model.best_estimator_.perplexity(Values))


# See model parameters
print(model.best_estimator_.get_params())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


Topic 0:
type object error string typescript time python value http argument
Topic 1:
error file core python module docker net server connection no
Topic 2:
react function app error flutter android component property swiftui method
Topic 3:
studio dart path package google keras project colab template code
Topic 4:
value flutter image material spring table text button data boot


Log Likelihood:  -176910.869008862
Perplexity:  6683.870918338985
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.9, 'learning_method': 'online', 'learning_offset': 50, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 5, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': 0, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


### 3.B) Negative Matrix Factorisation (NMF)

## 4) Keywords extraction 